# Preparação do corpus e pré-processamento

In [ ]:
from google.colab import files
import glob

arquivo = files.upload()
!unzip 'Enron.zip' -d 'enron'

In [ ]:
import nltk
nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')


def tokenizar(str_texto):
    return word_tokenize(str_texto)

def limpar(lista):
    return [i.lower() for i in lista if i.isalpha()]

def sem_stops(lista):
    return [i for i in lista if i not in stops]

def stemizar(lista):
    return [stemmer.stem(i) for i in lista]

In [ ]:
arqs = glob.glob('enron/*.txt')
mensagens = list()
etiquetas = list()
for arq in arqs:
    arquivo = open(arq, 'r')
    classe = int(arquivo.readline()[0])  # Pega só o número e deixa de fora o \n
    
    texto = arquivo.read()
    texto = stemizar(sem_stops(limpar(tokenizar(texto))))
    mensagens.append(texto)
    etiquetas.append(classe)
    arquivo.close()

In [ ]:
# Conferindo...
etiquetas[:10]

[0, 0, 0, 0, 0, 0, 0, 1, 0, 1]

# Codificação (encoding)

In [ ]:
import numpy as np

vocab = set([p for sent in mensagens for p in sent])

# Cria um dicionário {palavra: índice}
indices_de_palavras = {palavra: e+1 for e, palavra in enumerate(vocab)}  # e+1 para que o primeiro índice não seja 0, que é um pad

# Gera um vetor de índices de palavras para cada mensagem
vetores_msg = np.array([[indices_de_palavras[p] for p in d] for d in mensagens], dtype=object)
vetores_msg

array([list([10122, 11196, 26047, 6504, 19166, 15434, 27702, 36643, 11196, 32418, 886, 19166, 10558, 32418, 886, 8610, 493, 19019, 17879]),
       list([10122, 7339, 34511, 17524, 1053, 9760, 8338, 33655, 5777, 10083, 26517, 24228, 10960, 7339, 2775, 23854, 18133, 1710, 22551, 15344, 26719, 13696, 15344, 13936, 19499, 7339, 23854, 24228, 10960, 31518, 12955, 1545]),
       list([10122, 22498, 37772, 9076, 22436, 5146, 18412, 22056, 4783, 6439, 19883, 9953, 11419, 3791, 15013, 28293, 18412, 22056, 4783, 6439, 19883, 10122, 22498, 37772, 9076, 22436, 36968, 17867, 25383, 8238, 35722, 24147, 9076, 22498, 37772, 22498, 36279, 2129, 2971, 35722, 22436, 35476, 6769, 32358, 3763, 22436, 15434, 1402, 19126, 1545, 17961, 5146, 9953, 11419, 30330, 3791, 20864, 29732, 1537, 16248, 6799, 3763, 36883, 4740, 11715, 30599, 22973, 4906, 17867, 20864, 6769, 37892, 25299, 7725, 2003, 17867, 26515, 16832, 6138, 31672, 15434, 1402, 19126, 22436, 1545, 3560, 5146, 1537, 16248, 6439, 19883, 15554, 26432, 12

In [ ]:
# Conferindo...
print(len(vocab))
indices_de_palavras['viagra']

37892


3425

# Sua vez: conversão para vetores binários

In [ ]:
def binarizar(matriz_int, dim= #????):
    binarizado = np.zeros((len(matriz_int), dim))

    for e, vetor in enumerate(matriz_int):
        binarizado[e, vetor] = 1.

    return binarizado

# Conversão em binários dos testos das resenhas (variável X)
vetores_msg_bin = #???????

In [ ]:
# Conferindo...
print(vetores_msg_bin.shape)
vetores_msg_bin[0]

In [ ]:
# Binarização das etiquetas de classificação. Variável Y.

etiquetas_bin = #???????????

In [ ]:
# Conferindo...
etiquetas_bin[:10] 

# Sua vez: partição dos dados em treinamento / teste

In [ ]:
# Partição treinamento / teste
treino_x = vetores_msg_bin[:round(len(mensagens) * 0.8)]
teste_x = vetores_msg_bin[#???????]

treino_y = etiquetas_bin[#???????]
teste_y = etiquetas_bin[len(treino_y) + 1:]

# Criação do modelo de rede

Partição dos dados de treinamento em (1) validação e (2) treinamento parcial, tanto X (resenhas) quanto Y (etiquetas).

In [ ]:
valid_x = treino_x[:2000]
treino_x_parcial = treino_x[2000:]
valid_y = treino_y[:2000] 
treino_y_parcial = treino_y[2000:]

# Sua vez 

Determine os parâmetros faltantes na arquitetura do modelo: forma da camada de entrada e tamanho da camada de saída.

In [ ]:
from keras import models 
from keras import layers

modelo = models.Sequential() 
modelo.add(layers.Dense(16, activation='relu', input_shape=#???????))
modelo.add(layers.Dense(16, activation='relu'))
modelo.add(layers.Dense(#???????, activation='sigmoid'))

# Sua vez

Preencha os parâmetros para a compilação do modelo de acordo com a informação do comentário.

In [ ]:
# A função de perda escolhida é a entropia cruzada, boa para classificação probabilística,
# e binária, pois temos duas classes possíveis para as etiquetas das resenhas (pos. e neg.)
# O otimizador é o Adam, sempre uma boa escolha para PLN.
# O desempenho será avaliado pela acurácia ("acc").

modelo.compile(optimizer=#???????, 
              loss=#???????, 
              metrics=#???????)

Treinamento do modelo compilado.

In [ ]:
historia = modelo.fit(treino_x_parcial, 
                    treino_y_parcial, 
                    epochs=30, 
                    batch_size=512, 
                    validation_data=(valid_x, valid_y), 
                    verbose=0  # já que será gerado um gráfico, não é tão importante ver os números do aprendizado
                    )

Exibição da evolução da perda no treinamento e na validação.

In [ ]:
import matplotlib.pyplot as plt

dic_historia = historia.history  # dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
perda = dic_historia['loss'] 
perda_valid = dic_historia['val_loss']

acuracia = dic_historia['acc']
epocas = range(1, len(acuracia) + 1)

plt.plot(epocas, perda, 'bo', label='Perda no treinamento')  # “bo” = pontilhado azul
plt.plot(epocas, perda_valid, 'b', label='Perda na validação')  # “b” = linha contínua azul
plt.title('Perda no Treinamento e na Validação') 
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()

plt.show()

Exibição da evolução da acurácia no teste e na validação.

In [ ]:
plt.clf()  # Limpa a figura

acuracia_treino = dic_historia['acc'] 
acuracia_valid = dic_historia['val_acc']

plt.plot(epocas, acuracia_treino, 'bo', label='Acurácia no Treinamento') 
plt.plot(epocas, acuracia_valid, 'b', label='Acurácia na Validação')
plt.title('Acurácia no Treinamento e na Validação') 
plt.xlabel('Épocas') 
plt.ylabel('Acurácia') 
plt.legend()

plt.show()

# Treinamento com `EarlyStopping`

O modelo será treinado novamente com um número grande de épocas, em busca de obter seu melhor desempenho e parar quando encontrá-lo "automaticamente", isto é, quando parar de ter melhoria no aprendizado.


In [ ]:
from tensorflow.keras import callbacks

aprendeu_parou = callbacks.EarlyStopping(
    min_delta=0.001,  # aprendizado mínimo (resultados menores não contarão como aprendizado)
    patience=10,  # por quantas épocas insistir?
    restore_best_weights=True,
)

historia = modelo.fit(treino_x_parcial, 
                    treino_y_parcial, 
                    epochs=300, 
                    batch_size=512, 
                    validation_data=(valid_x, valid_y),
                    callbacks=[aprendeu_parou])

# Sua vez

Preencha os parâmetros da função de avaliação. 

In [ ]:
avaliacao = modelo.evaluate(#???????, #???????)

In [ ]:
print('Acurácia na avaliação: ', avaliacao[1], '\nPerda: ', avaliacao[0])

# Playground!

Experimente mudar a arquitetura da rede (tamanho e profundidade de camadas) para ver se são produzidas diferenças nos resultados.

E que tal mudar:

* A função de perda para `mse` (erro quadrático médio)
* A função de ativação para `tanh` (tangente hiperbólica) ao invés de `relu`

Use `EarlyStopping` para limitar o número de épocas de aprendizado.